<a href="https://colab.research.google.com/github/dinuka-rp/nerdy-panda/blob/main/Prototype/RecSys/Basic_Content_based_NFT_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Content-based NFT Recommender System

https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

Check this later (not that different): https://medium.com/dataseries/recommendation-system-in-python-4d4cf6d33166 , https://gist.github.com/deansublett/06f010a886831f9dac5b1f9bce4f4229?source=post_page---------------------------

In [ ]:
!pip install rake-nltk

In [ ]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Get Dataset

In [ ]:
from google.colab import drive
drive.mount('drive')

assets_file_path = "/content/drive/MyDrive/Datasets/FYP/collected-nft-assets"
df = pd.read_csv(assets_file_path, sep='\t')

# df = df[['nft_id','traits_string','asset_contract_address', 'total_rarity']]
df.head()

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


,nft_id,name,asset_description,asset_contract_address,collection_name,collection_description,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url
0,2541585283815458503018281017495372409420154545...,CelebCard #84 Snoop Dogg,NaN,0x495f947276749ce646f68ac8c248420045cb7b5e,Mesmerizer,No theme. No traits. No rules. A treasure trov...,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...
1,4460258260744616717349398633449740654585409544...,3D Penguin Planet Official Collection,3D Penguin Planet Collection It was hand paint...,0x495f947276749ce646f68ac8c248420045cb7b5e,3D Penguin Planet Official Collection,3D Penguin Planet Official Collection It was h...,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...
2,1143238369460653929904223751867794581426030911...,VECTOR KAS 10 (The Relax),VECTOR KAS 10,0x495f947276749ce646f68ac8c248420045cb7b5e,Kalvintaka Collection,NaN,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...
3,5742295951199733757787373026863398866922647154...,Gooniez #1989,"A gang of 8,888 gooniez unleashed to wreak hav...",0x495f947276749ce646f68ac8c248420045cb7b5e,Metaverse GooniezNFT,"A gang of 8,888 gooniez unleashed to wreak hav...",metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...
4,1006254684714400059060824656421626037946389019...,Darumaa Soul Story.#3,Works that are created by an Extraterrestrial ...,0x495f947276749ce646f68ac8c248420045cb7b5e,Extraterrestrial Soul Darumaa,Darumaa. Not Real & Reality.\n\nWorks that are...,darumaa,False,FRABATO,0xde77ff31b3601243b0f41441e1f015456b2b1ad2,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/9xcm5vesaQYE...


## Data Cleaning

In [ ]:
for index, row in df.iterrows():
    df.at[index,'reference_id'] = row["asset_contract_address"] + "-" + str(row["nft_id"])

# df['traits_string'] = df['traits_string'].str.replace(';',' ') # this needs to be changed in BAYC dataset to ;;

df.head()

,nft_id,name,asset_description,asset_contract_address,collection_name,collection_description,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url,reference_id
0,2541585283815458503018281017495372409420154545...,CelebCard #84 Snoop Dogg,NaN,0x495f947276749ce646f68ac8c248420045cb7b5e,Mesmerizer,No theme. No traits. No rules. A treasure trov...,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...,0x495f947276749ce646f68ac8c248420045cb7b5e-254...
1,4460258260744616717349398633449740654585409544...,3D Penguin Planet Official Collection,3D Penguin Planet Collection It was hand paint...,0x495f947276749ce646f68ac8c248420045cb7b5e,3D Penguin Planet Official Collection,3D Penguin Planet Official Collection It was h...,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...,0x495f947276749ce646f68ac8c248420045cb7b5e-446...
2,1143238369460653929904223751867794581426030911...,VECTOR KAS 10 (The Relax),VECTOR KAS 10,0x495f947276749ce646f68ac8c248420045cb7b5e,Kalvintaka Collection,NaN,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...,0x495f947276749ce646f68ac8c248420045cb7b5e-114...
3,5742295951199733757787373026863398866922647154...,Gooniez #1989,"A gang of 8,888 gooniez unleashed to wreak hav...",0x495f947276749ce646f68ac8c248420045cb7b5e,Metaverse GooniezNFT,"A gang of 8,888 gooniez unleashed to wreak hav...",metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...,0x495f947276749ce646f68ac8c248420045cb7b5e-574...
4,1006254684714400059060824656421626037946389019...,Darumaa Soul Story.#3,Works that are created by an Extraterrestrial ...,0x495f947276749ce646f68ac8c248420045cb7b5e,Extraterrestrial Soul Darumaa,Darumaa. Not Real & Reality.\n\nWorks that are...,darumaa,False,FRABATO,0xde77ff31b3601243b0f41441e1f015456b2b1ad2,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/9xcm5vesaQYE...,0x495f947276749ce646f68ac8c248420045cb7b5e-100...


In [ ]:
df.iloc[0]

nft_id                            2541585283815458503018281017495372409420154545...
name                                                       CelebCard #84 Snoop Dogg
asset_description                                                               NaN
asset_contract_address                   0x495f947276749ce646f68ac8c248420045cb7b5e
collection_name                                                          Mesmerizer
collection_description            No theme. No traits. No rules. A treasure trov...
collection_slug                                                          mesmerizer
collection_verification                                                       False
creator_username                                                                NaN
creator_address                          0x3830dce8b8f5543404e005776b4de22870c01d64
creator_profile_img_url           https://storage.googleapis.com/opensea-static/...
owner_username                                                          Null

## Rake Vectorizer
Vectorize words

In [ ]:
def vectorize_words_in_column( column_name ):

  # initializing the new column
  new_column_name = 'Key_words_' + column_name
  df[new_column_name] = ""

  for index, row in df.iterrows():
      column_cell = row[column_name]

      
      # check if the cell has a string, to apply NLP
      if isinstance(column_cell, str):
        # instantiating Rake, by default it uses english stopwords from NLTK
        # and discards all puntuation characters as well
        r = Rake()

        # extracting the words by passing the text
        r.extract_keywords_from_text(column_cell)

        # getting the dictionary with key words as keys and their scores as values
        key_words_dict_scores = r.get_word_degrees()

        # print(key_words_dict_scores.keys())

        # assigning the key words to the new column for the corresponding movie
        # row[new_column_name] = list(key_words_dict_scores.keys()) # this doesn't work for some reason

        df.at[index, new_column_name]=list(key_words_dict_scores.keys())

      else:
        df.at[index, new_column_name] = None

  # dropping the column_name column
  df.drop(columns = [column_name], inplace = True)


In [ ]:
vectorize_words_in_column('name')
vectorize_words_in_column('asset_description')
vectorize_words_in_column('collection_name')
vectorize_words_in_column('collection_description')

df.head()

,nft_id,asset_contract_address,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url,reference_id,Key_words_name,Key_words_asset_description,Key_words_collection_name,Key_words_collection_description
0,2541585283815458503018281017495372409420154545...,0x495f947276749ce646f68ac8c248420045cb7b5e,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...,0x495f947276749ce646f68ac8c248420045cb7b5e-254...,"[celebcard, 84, snoop, dogg]",None,[mesmerizer],"[theme, traits, rules, treasure, trove, pure, ..."
1,4460258260744616717349398633449740654585409544...,0x495f947276749ce646f68ac8c248420045cb7b5e,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...,0x495f947276749ce646f68ac8c248420045cb7b5e-446...,"[3d, penguin, planet, official, collection]","[3d, penguin, planet, collection, hand, painte...","[3d, penguin, planet, official, collection]","[3d, penguin, planet, official, collection, ha..."
2,1143238369460653929904223751867794581426030911...,0x495f947276749ce646f68ac8c248420045cb7b5e,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...,0x495f947276749ce646f68ac8c248420045cb7b5e-114...,"[vector, kas, 10, relax]","[vector, kas, 10]","[kalvintaka, collection]",None
3,5742295951199733757787373026863398866922647154...,0x495f947276749ce646f68ac8c248420045cb7b5e,metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...,0x495f947276749ce646f68ac8c248420045cb7b5e-574...,"[gooniez, 1989]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[metaverse, goonieznft]","[gang, 8, 888, gooniez, unleashed, wreak, havo..."
4,1006254684714400059060824656421626037946389019...,0x495f947276749ce646f68ac8c248420045cb7b5e,darumaa,False,FRABATO,0xde77ff31b3601243b0f41441e1f015456b2b1ad2,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/9xcm5vesaQYE...,0x495f947276749ce646f68ac8c248420045cb7b5e-100...,"[darumaa, soul, story, .#, 3]","[works, created, extraterrestrial, soul, named...","[extraterrestrial, soul, darumaa]","[darumaa, real, reality, works, created, extra..."


### Combine all key words into one column

This may come handy when comparing with trends data.

In [ ]:
columns = df.columns
df['All_key_words_list'] = pd.NA
df['All_key_words_str'] = ''

for index, row in df.iterrows():
  words = []
  key_words_str=''

  for col in columns:
    if col.startswith("Key_words") and row[col] != None:
      words += row[col]
      key_words_str += ' '.join(row[col])+ ' '

  # row['All_key_words_list'] = words
  # row['All_key_words_str'] = key_words_str
  df.at[index, 'All_key_words_list'] = words
  df.at[index, 'All_key_words_str'] = key_words_str

df.head()

,nft_id,asset_contract_address,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url,reference_id,Key_words_name,Key_words_asset_description,Key_words_collection_name,Key_words_collection_description,All_key_words_list,All_key_words_str
0,2541585283815458503018281017495372409420154545...,0x495f947276749ce646f68ac8c248420045cb7b5e,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...,0x495f947276749ce646f68ac8c248420045cb7b5e-254...,"[celebcard, 84, snoop, dogg]",None,[mesmerizer],"[theme, traits, rules, treasure, trove, pure, ...","[celebcard, 84, snoop, dogg, mesmerizer, theme...",celebcard 84 snoop dogg mesmerizer theme trait...
1,4460258260744616717349398633449740654585409544...,0x495f947276749ce646f68ac8c248420045cb7b5e,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...,0x495f947276749ce646f68ac8c248420045cb7b5e-446...,"[3d, penguin, planet, official, collection]","[3d, penguin, planet, collection, hand, painte...","[3d, penguin, planet, official, collection]","[3d, penguin, planet, official, collection, ha...","[3d, penguin, planet, official, collection, 3d...",3d penguin planet official collection 3d pengu...
2,1143238369460653929904223751867794581426030911...,0x495f947276749ce646f68ac8c248420045cb7b5e,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...,0x495f947276749ce646f68ac8c248420045cb7b5e-114...,"[vector, kas, 10, relax]","[vector, kas, 10]","[kalvintaka, collection]",None,"[vector, kas, 10, relax, vector, kas, 10, kalv...",vector kas 10 relax vector kas 10 kalvintaka c...
3,5742295951199733757787373026863398866922647154...,0x495f947276749ce646f68ac8c248420045cb7b5e,metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...,0x495f947276749ce646f68ac8c248420045cb7b5e-574...,"[gooniez, 1989]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[metaverse, goonieznft]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[gooniez, 1989, gang, 8, 888, gooniez, unleash...",gooniez 1989 gang 8 888 gooniez unleashed wrea...
4,1006254684714400059060824656421626037946389019...,0x495f947276749ce646f68ac8c248420045cb7b5e,darumaa,False,FRABATO,0xde77ff31b3601243b0f41441e1f015456b2b1ad2,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/9xcm5vesaQYE...,0x495f947276749ce646f68ac8c248420045cb7b5e-100...,"[darumaa, soul, story, .#, 3]","[works, created, ext

Create `bag_of_words`

Drop useless columns

In [ ]:
df['All_key_words_str'].count()

10100

In [ ]:
df.set_index('reference_id', inplace = True)   # set reference_id as the index of the dataframe
df.head()

,nft_id,asset_contract_address,collection_slug,collection_verification,creator_username,creator_address,creator_profile_img_url,owner_username,owner_address,owner_profile_img_url,traits_string,total_rarity,num_sales,last_sale_timestamp,last_sale_total_price,last_sale_from_account_address,top_bid,open_sea_link,image_url,Key_words_name,Key_words_asset_description,Key_words_collection_name,Key_words_collection_description,All_key_words_list,All_key_words_str
reference_id,,,,,,,,,,,,,,,,,,,,,,,,,
0x495f947276749ce646f68ac8c248420045cb7b5e-25415852838154585030182810174953724094201545459452816665085826154914734145537,2541585283815458503018281017495372409420154545...,0x495f947276749ce646f68ac8c248420045cb7b5e,mesmerizer,False,NaN,0x3830dce8b8f5543404e005776b4de22870c01d64,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/DwknciiKsguS...,"[celebcard, 84, snoop, dogg]",None,[mesmerizer],"[theme, traits, rules, treasure, trove, pure, ...","[celebcard, 84, snoop, dogg, mesmerizer, theme...",celebcard 84 snoop dogg mesmerizer theme trait...
0x495f947276749ce646f68ac8c248420045cb7b5e-44602582607446167173493986334497406545854095441469804519114872108262954631169,4460258260744616717349398633449740654585409544...,0x495f947276749ce646f68ac8c248420045cb7b5e,3d-penguin-planet-official-collection,False,NaN,0x629c2ac970da51a3fa29af4015e06c250c267e36,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/yzniUTh85SYY...,"[3d, penguin, planet, official, collection]","[3d, penguin, planet, collection, hand, painte...","[3d, penguin, planet, official, collection]","[3d, penguin, planet, official, collection, ha...","[3d, penguin, planet, official, collection, 3d...",3d penguin planet official collection 3d pengu...
0x495f947276749ce646f68ac8c248420045cb7b5e-114323836946065392990422375186779458142603091121322655546213463221980662595585,1143238369460653929904223751867794581426030911...,0x495f947276749ce646f68ac8c248420045cb7b5e,kalvintaka-collection,False,Creative_FineArts_and_Designs,0xfcc0ffa7b48f9af939ed6e9d3c535e371ab2f17f,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/2xDLW3Nrrt8u...,"[vector, kas, 10, relax]","[vector, kas, 10]","[kalvintaka, collection]",None,"[vector, kas, 10, relax, vector, kas, 10, kalv...",vector kas 10 relax vector kas 10 kalvintaka c...
0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444570123223695361,5742295951199733757787373026863398866922647154...,0x495f947276749ce646f68ac8c248420045cb7b5e,metaverse-goonieznft,False,GooniezNFTCollection,0x7ef43e4a20315830feacb23fe17f7cf2d49f2491,https://storage.googleapis.com/opensea-static/...,NullAddress,0x0000000000000000000000000000000000000000,https://storage.googleapis.com/opensea-static/...,NaN,0.0,0,NaN,NaN,NaN,NaN,https://opensea.io/assets/0x495f947276749ce646...,https://lh3.googleusercontent.com/Y4lDr3FzA7xm...,"[gooniez, 1989]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[metaverse, goonieznft]","[gang, 8, 888, gooniez, unleashed, wreak, havo...","[gooniez, 1989, gang, 8, 888, gooniez, unleash...",gooniez 1989 gang 8 888 gooniez unleashed wrea...
0x495f947276749ce646f68ac8c248420045cb7b5e-100625468471440005906082465642162603794638901987049655284968819362997051850753,1006254684714400059060824656421626037946389019...,0x495f947276749ce646f68ac8c248420045cb7b5e,darumaa,False,FRABATO,0xde7

In [ ]:
df.drop(columns = [col for col in df.columns if not col.startswith("All_key_words")], inplace = True)
df.head()

,All_key_words_list,All_key_words_str
reference_id,,
0x495f947276749ce646f68ac8c248420045cb7b5e-25415852838154585030182810174953724094201545459452816665085826154914734145537,"[celebcard, 84, snoop, dogg, mesmerizer, theme...",celebcard 84 snoop dogg mesmerizer theme trait...
0x495f947276749ce646f68ac8c248420045cb7b5e-44602582607446167173493986334497406545854095441469804519114872108262954631169,"[3d, penguin, planet, official, collection, 3d...",3d penguin planet official collection 3d pengu...
0x495f947276749ce646f68ac8c248420045cb7b5e-114323836946065392990422375186779458142603091121322655546213463221980662595585,"[vector, kas, 10, relax, vector, kas, 10, kalv...",vector kas 10 relax vector kas 10 kalvintaka c...
0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444570123223695361,"[gooniez, 1989, gang, 8, 888, gooniez, unleash...",gooniez 1989 gang 8 888 gooniez unleashed wrea...
0x495f947276749ce646f68ac8c248420045cb7b5e-100625468471440005906082465642162603794638901987049655284968819362997051850753,"[darumaa, soul, story, .#, 3, works, created, ...",darumaa soul story .# 3 works created extrater...


In [ ]:
df.iloc[0]

nft_id                              2541585283815458503018281017495372409420154545...
asset_contract_address                     0x495f947276749ce646f68ac8c248420045cb7b5e
collection_slug                                                            mesmerizer
collection_verification                                                         False
creator_username                                                                  NaN
creator_address                            0x3830dce8b8f5543404e005776b4de22870c01d64
creator_profile_img_url             https://storage.googleapis.com/opensea-static/...
owner_username                                                            NullAddress
owner_address                              0x0000000000000000000000000000000000000000
owner_profile_img_url               https://storage.googleapis.com/opensea-static/...
traits_string                                                                     NaN
total_rarity                                          

## Modeling

In order to detect similarities between movies, I need to vectorize, as I mentioned above. I decided to use CountVectorizer rather than TfIdfVectorizer for one simple reason: I need a simple frequency counter for each word in my bag_of_words column. Tf-Idf tends to give less importance to the words that are more present in the entire corpus (our whole column, in this case) which is not what we want for this application, because every word is important to detect similarity! 

*It seems like Tf-Idf would give more meaningful results* for ranking - check this later. For relevance, CountVectorizer seems to be ok.

Once I have the matrix containing the count for each word, we can apply the cosine_similarity function

In [ ]:
# instantiating and generating the count matrix
count = CountVectorizer()   # used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text
count_matrix = count.fit_transform(df['All_key_words_str'])

# print(count_matrix)
count_matrix.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
count_matrix.shape

(10100, 7529)

in the count_matrix, i seems to be the movie, j seems to be each word in the bag_of_words per movie. the value at i, j gives the frequency of each word in the entire column of bag_of_words.

For some reason (need to find out) this frequency is needed to calculate the cosine similarity between the content in movies. - *The definition of similarity between two vectors u and v is, in fact, the ratio between their dot product and the product of their magnitudes.*

In [ ]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes - easy to refer the movie title by index from the array that is created (cosine_sim)
indices = pd.Series(df.index)
indices[:5]

0    0x495f947276749ce646f68ac8c248420045cb7b5e-254...
1    0x495f947276749ce646f68ac8c248420045cb7b5e-446...
2    0x495f947276749ce646f68ac8c248420045cb7b5e-114...
3    0x495f947276749ce646f68ac8c248420045cb7b5e-574...
4    0x495f947276749ce646f68ac8c248420045cb7b5e-100...
Name: reference_id, dtype: object

Check if this concern can be addressed: Having cosine_sim in-memory can be a memory constraint for larger datasets

In [ ]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
# cosine_sim.todense()

In [ ]:
# function that takes in movie title (nft_contract_address-nft_id) as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # getting the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

## Testing

In [ ]:
recommendations('0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444570123223695361')

['0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444570123223695361',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444682273409728513',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444687770967867393',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444669079270195201',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444641591479500801',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444656984642289665',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444666880246939649',
 '0x495f947276749ce646f68ac8c248420045cb7b5e-57422959511997337577873730268633988669226471548944456734432444699865595772929',


# Custom Integration to rerank using Social Trends

Phrase-Based Matching

In [ ]:
trending_phrases = ['ROMCOM SERIES FOR HAESOO', 'Golden Corral', 'Wordle 228', 'Tietê', 'BOP TRACKLIST', '伊東純也', 'JIKJIN FOCUS FILM', 'NINJAPAN 2021', '一気読み', 'ムササビ', 'オーストラリア', 'WONPIL Concept Photo 3', 'Benzine 39', 'あーりん', 'البليهي', 'Kraft', '花枝ちゃん', 'HAPPY BIRTHDAY GEONU', 'ちゃん誕生日', 'サウジアラビア', '도영듀스', '凛ちゃん', 'SECRETCRUSH TRAILER', 'Best Entertainer Jungkook', 'UM CORPO DE VERÃO', 'Quito', 'Damn Brady', 'Schefter', '日村さん', 'SMILE FOR MEW', 'HBCUs', 'もやとらっしー']
# phrases contains all trending words

In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

In [ ]:
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)

# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in trending_phrases]
matcher.add("TerminologyList", patterns)


Content-based matching